In [1]:
# TODO: check that my optimizer finds qubit codes that agree with Vlad's mathematica script 

# TODO: optimize functions. eg. remove allocations, remove repeated computations, try reduce # for loops. 
#           - rules is extremely fast (50ns). 
#           - rules4_5 still relatively slow (7μs). Its basically non-allocating, so I don't immedaitely see how to speed it up. 

# TODO: try and find qutrit solutions 

using Pkg
using LinearAlgebra
using Optim 
using LineSearches
using Combinatorics
using PyPlot
using BenchmarkTools
using JLD2

In [2]:
n = 2; # number of physical qudits 
q = 2; # physical qudit dimension 
d = 2; # logical qudit dimension

function partitions_into_q_parts(n, q)
    # Start with an array of q parts filled with zeros
    partitions = zeros(Int64,0,q)
    
    # Recursive function to generate partitions
    function generate_partition(partition, remaining, parts_left)
        # Base case: if no more parts are left and no remaining value, store partition
        if parts_left == 0
            if remaining == 0
                partitions = vcat(partitions, copy(partition))
            end
            return
        end
        
        # Loop to assign values to the current part, ranging from 0 to the remaining value
        for i in 0:remaining
            partition[parts_left] = i
            generate_partition(partition, remaining - i, parts_left - 1)
        end
    end
    
    # Initialize the partition array
    generate_partition(zeros(Int, 1,q), n, q)
        
    return partitions
end;

heaviside(x) = x ≥ 0 ? 1 : 0;
checknonneg(λ) = all(x -> x ≥ 0, λ);
partbinom(n,λ) = (checknonneg(λ) == 1) ? (factorial(n)/prod(factorial(λ[i]) for i in eachindex(λ))) : 0;

function rules1(n,q,d,codewords)

    local_num_var_params = length(codewords)
    local_codeword_length = Int(local_num_var_params/d)

    outval = 0 
    @inbounds for i in 1:d 
        @inbounds for j in 1:d 
            if i ≥ j 
                @views outval += abs( abs(codewords[(1+(i-1)*local_codeword_length):local_codeword_length*i]' * codewords[(1+(j-1)*local_codeword_length):local_codeword_length*j]) - (i==j) )
            end
        end
    end
    return outval

end

rule4binoms_diff(n,t,λ,λ_minus_μ,λ_minus_μ_plus_ν) = ( checknonneg(λ_minus_μ) == 1 ? partbinom(n-2t,λ_minus_μ)/( sqrt(partbinom(n,λ)*partbinom(n,λ_minus_μ_plus_ν)) ) : 0 )

function x_min_y!(out,x,y)
    for i in eachindex(x)
        out[i] = x[i] - y[i] 
    end 
end;
function x_min_y_plus_z!(out,x,y,z)
    for i in eachindex(x)
        out[i] = x[i] - y[i] + z[i]
    end 
end;

function binoms_diff_precompute(n, q, t, λ, μ, ν, vλ, vμ, vν)
    
    λ_minus_μ = zeros(Int, q)
    λ_minus_μ_plus_ν = zeros(Int, q)
    
    binoms_diff = zeros(Float64,vμ,vν,vλ)
    for k in 1:vμ
        μview = view(μ, k, :)
        for l in 1:vν
            νview = view(ν, l, :)
            for iλ in 1:vλ
                λview = view(λ, iλ, :)
                
                x_min_y!(λ_minus_μ, λview, μview)
                x_min_y_plus_z!(λ_minus_μ_plus_ν, λview, μview, νview)
                
                if checknonneg(λ_minus_μ) != 0 && checknonneg(λ_minus_μ_plus_ν) != 0
                    binoms_diff[k,l,iλ] = rule4binoms_diff(n, t, λview, λ_minus_μ, λ_minus_μ_plus_ν)
                end
            end
            
        end
    end
    return binoms_diff
    
end

function nonneg_precompute(q, λ, μ, ν, vλ, vμ, vν)

    λ_minus_μ = zeros(Int, q)
    λ_minus_μ_plus_ν = zeros(Int, q)
        
    nonneg_matr = zeros(Int64,vμ,vν,vλ)
    for k in 1:vμ
        μview = view(μ, k, :)  
        for l in 1:vν
            νview = view(ν, l, :)
            
            for iλ in 1:vλ
                λview = view(λ, iλ, :)
                
                x_min_y!(λ_minus_μ, λview, μview)
                x_min_y_plus_z!(λ_minus_μ_plus_ν, λview, μview, νview)
                
                if checknonneg(λ_minus_μ) != 0 && checknonneg(λ_minus_μ_plus_ν) != 0
                    nonneg_matr[k,l,iλ] = 1
                end
            end
            
        end
    end
    
    return nonneg_matr

end


function rule4_5(n, q, d, t, codewords, λ, μ, ν, vλ, vμ, vν)

    # Preallocate outside loops
    local_num_var_params = length(codewords)
    local_codeword_length = Int(local_num_var_params/d)
    outval1 = 0.0
    outval2 = 0.0
        
    @inbounds for i in 1:d-1
        @views codewords_i = codewords[(1+(i-1)*local_codeword_length):local_codeword_length*i]
            
        for j in i+1:d
            @views codewords_j = codewords[(1+(j-1)*local_codeword_length):local_codeword_length*j]
            
            for k in 1:vμ
                for l in 1:vν
                    λsum_rule4 = 0.0
                    λsum_rule5 = 0.0
                    
                    for iλ in 1:vλ                        
                        if nonneg_cached[k,l,iλ] != 0

                            code_i_iλ = codewords_i[iλ]
                            code_j_iλ_k_l = codewords_j[iλ - k + l]
                            λsum_rule4 += code_i_iλ' * code_j_iλ_k_l * binoms_diff_cached[k,l,iλ]
                            
                            code_i_iλ_k_l = codewords_i[iλ - k + l]
                            code_j_iλ = codewords_j[iλ]
                            λsum_rule5 += (code_i_iλ' * code_i_iλ_k_l - code_j_iλ' * code_j_iλ_k_l) * binoms_diff_cached[k,l,iλ]

                        end
                    end  
                    outval1 += abs(λsum_rule4)
                    outval2 += abs(λsum_rule5)
                end
            end
        end
    end
    
    return abs(outval1) + abs(outval2)

end

rule4_5 (generic function with 1 method)

In [7]:
# check n = 7 qubit Ruskai codes 
n = 7; 
q = 2; 
d = 2; 
t = 1;
λ = partitions_into_q_parts(n,q);
μ = partitions_into_q_parts(2t,q);
ν = partitions_into_q_parts(2t,q);
vλ = size(λ)[1];
vμ = size(μ)[1];
vν = size(ν)[1];
binoms_diff_cached = binoms_diff_precompute(n, q, t, λ, μ, ν, vλ, vμ, vν); # TODO: make const 
nonneg_cached = nonneg_precompute(q, λ, μ, ν, vλ, vμ, vν); # TODO: make const 
ruskai_codewords_c0 = [sqrt(15);0;-sqrt(7);0;sqrt(21);0;sqrt(21);0]/8;
ruskai_codewords_c1 = [0;sqrt(21);0;sqrt(21);0;-sqrt(7);0;sqrt(15)]/8;
codewords = [ruskai_codewords_c0;ruskai_codewords_c1];

@show rules1(n,q,d,codewords)
@show rule4_5(n,q,d,t,codewords,λ,μ,ν,vλ,vμ,vν)

rules1(n, q, d, codewords) = 0.0
rule4_5(n, q, d, t, codewords, λ, μ, ν, vλ, vμ, vν) = 7.112366251504909e-17


7.112366251504909e-17

In [21]:
### ---------------------- Optimisation Code ---------------------- ###

n = 19;
q = 2;
d = 2;
t = 2;
λ = partitions_into_q_parts(n,q);
μ = partitions_into_q_parts(2t,q);
ν = partitions_into_q_parts(2t,q);
vλ = size(λ)[1];
vμ = size(μ)[1];
vν = size(ν)[1];
binoms_diff_cached = binoms_diff_precompute(n, q, t, λ, μ, ν, vλ, vμ, vν);
nonneg_cached = nonneg_precompute(q, λ, μ, ν, vλ, vμ, vν);

In [20]:
binoms_diff_cached;

In [18]:
# Define cost function  
cost(x) = abs(rules1(n,q,d,x)) + rule4_5(n,q,d,t,x,λ,μ,ν,vλ,vμ,vν)

cost (generic function with 1 method)

In [15]:
# Initial guess 
codeword_length = size(partitions_into_q_parts(n,q))[1]
num_var_params = d * codeword_length
x0 = rand(num_var_params)
cost(x0)

# Optimize using LBFGS 
res_loop_minimum = []; res_loop_minimizer = [];
for i in 1:5
    # x0 = [normalize(rand(n+1));normalize(rand(n+1))];
    x0 = rand(num_var_params)
    res = optimize(cost, x0, LBFGS(linesearch=LineSearches.BackTracking()),
    # res = optimize(cost, x0, NelderMead(),
                Optim.Options(iterations=50000,
                            g_tol=1e-8,
                            allow_f_increases=true,
                            show_trace=true)
                    )
    println("iteration $i: min val = $(res.minimum)")
    push!(res_loop_minimum, res.minimum)
    push!(res_loop_minimizer, res.minimizer )
end

minval,minloc = findmin(res_loop_minimum)
minx0 = res_loop_minimizer[minloc]

Iter     Function value   Gradient norm 
     0     2.909979e+01     6.257101e+00
 * time: 0.0
     1     4.034906e+00     3.668929e+00
 * time: 0.006999969482421875
     2     3.364850e+00     2.807416e+00
 * time: 0.016000032424926758
     3     1.670083e+00     9.695809e-01
 * time: 0.04900002479553223
     4     1.660106e+00     9.463511e-01
 * time: 0.05800008773803711
     5     1.424879e+00     1.014960e+00
 * time: 0.06599998474121094
     6     1.179451e+00     1.516492e+00
 * time: 0.07400012016296387
     7     1.121546e+00     2.074481e+00
 * time: 0.0820000171661377
     8     8.587172e-01     1.143871e+00
 * time: 0.08999991416931152
     9     7.486544e-01     1.194213e+00
 * time: 0.0989999771118164
    10     7.412057e-01     1.965830e+00
 * time: 0.10700011253356934
    11     6.832770e-01     1.176221e+00
 * time: 0.11700010299682617
    12     6.403235e-01     1.481755e+00
 * time: 0.12299990653991699
    13     6.347248e-01     2.301446e+00
 * time: 0.1400001049041

   118     2.876140e-01     1.170502e+00
 * time: 0.9779999256134033
   119     2.875327e-01     2.034536e+00
 * time: 0.9860000610351562
   120     2.874539e-01     1.799251e+00
 * time: 0.9939999580383301
   121     2.873497e-01     1.131435e+00
 * time: 1.003000020980835
   122     2.872848e-01     2.322728e+00
 * time: 1.0109999179840088
   123     2.871707e-01     9.729044e-01
 * time: 1.0190000534057617
   124     2.871660e-01     9.738934e-01
 * time: 1.0280001163482666
   125     2.871262e-01     9.728706e-01
 * time: 1.0360000133514404
   126     2.871198e-01     1.713318e+00
 * time: 1.0439999103546143
   127     2.870946e-01     1.669787e+00
 * time: 1.0529999732971191
   128     2.870574e-01     1.206519e+00
 * time: 1.061000108718872
   129     2.869880e-01     1.567235e+00
 * time: 1.0740001201629639
   130     2.869816e-01     1.819030e+00
 * time: 1.0859999656677246
   131     2.869673e-01     1.328935e+00
 * time: 1.0959999561309814
   132     2.869592e-01     7.148292

   238     2.624232e-01     5.696202e-01
 * time: 1.9749999046325684
   239     2.624232e-01     9.953011e-01
 * time: 1.9830000400543213
   240     2.624208e-01     1.020103e+00
 * time: 1.9920001029968262
   241     2.624163e-01     8.621590e-01
 * time: 2.0
   242     2.623999e-01     7.285883e-01
 * time: 2.007999897003174
   243     2.623934e-01     1.165190e+00
 * time: 2.0190000534057617
   244     2.623651e-01     1.086378e+00
 * time: 2.0250000953674316
   245     2.622774e-01     2.202281e+00
 * time: 2.0329999923706055
   246     2.622034e-01     1.420129e+00
 * time: 2.0410001277923584
   247     2.620868e-01     1.419680e+00
 * time: 2.049999952316284
   248     2.619391e-01     1.717269e+00
 * time: 2.058000087738037
   249     2.619254e-01     1.149163e+00
 * time: 2.066999912261963
   250     2.619179e-01     7.651595e-01
 * time: 2.075000047683716
   251     2.619128e-01     5.625178e-01
 * time: 2.0840001106262207
   252     2.619023e-01     1.857659e+00
 * time: 2.09

   358     2.538070e-01     1.671834e+00
 * time: 2.888000011444092
   359     2.538026e-01     1.603915e+00
 * time: 2.8959999084472656
   360     2.537845e-01     1.371066e+00
 * time: 2.8959999084472656
   361     2.537471e-01     1.164208e+00
 * time: 2.9049999713897705
   362     2.537315e-01     1.078756e+00
 * time: 2.9140000343322754
   363     2.536606e-01     1.377425e+00
 * time: 2.9230000972747803
   364     2.536183e-01     1.377397e+00
 * time: 2.928999900817871
   365     2.535957e-01     1.117798e+00
 * time: 2.937000036239624
   366     2.535698e-01     1.299028e+00
 * time: 2.946000099182129
   367     2.535127e-01     1.165507e+00
 * time: 2.9539999961853027
   368     2.534970e-01     9.497660e-01
 * time: 2.9609999656677246
   369     2.534888e-01     1.598013e+00
 * time: 2.9679999351501465
   370     2.534299e-01     1.640059e+00
 * time: 2.9739999771118164
   371     2.534068e-01     1.131169e+00
 * time: 2.9779999256134033
   372     2.533849e-01     9.178934e-

   479     2.506340e-01     1.868494e+00
 * time: 3.7939999103546143
   480     2.506181e-01     6.060621e-01
 * time: 3.802000045776367
   481     2.506120e-01     8.996741e-01
 * time: 3.811000108718872
   482     2.506058e-01     9.253564e-01
 * time: 3.822999954223633
   483     2.505950e-01     8.623746e-01
 * time: 3.8269999027252197
   484     2.505895e-01     1.195066e+00
 * time: 3.8389999866485596
   485     2.505781e-01     9.921074e-01
 * time: 3.8440001010894775
   486     2.505667e-01     9.152816e-01
 * time: 3.8519999980926514
   487     2.505331e-01     1.155786e+00
 * time: 3.8600001335144043
   488     2.504902e-01     7.265263e-01
 * time: 3.8610000610351562
   489     2.504850e-01     1.410473e+00
 * time: 3.877000093460083
   490     2.504796e-01     1.410450e+00
 * time: 3.884999990463257
   491     2.504618e-01     1.410458e+00
 * time: 3.884999990463257
   492     2.504436e-01     1.282988e+00
 * time: 3.8940000534057617
   493     2.504379e-01     8.113046e-01

...

   622     1.169366e-01     3.792624e-01
 * time: 4.663000106811523
   623     1.169364e-01     5.120502e-01
 * time: 4.663000106811523
   624     1.169345e-01     6.149291e-01
 * time: 4.671000003814697
   625     1.169281e-01     4.040071e-01
 * time: 4.67900013923645
   626     1.169220e-01     5.423555e-01
 * time: 4.687000036239624
   627     1.169188e-01     1.271652e+00
 * time: 4.700000047683716
   628     1.169062e-01     7.799502e-01
 * time: 4.703000068664551
   629     1.169030e-01     9.416484e-01
 * time: 4.714000225067139
   630     1.168987e-01     1.005118e+00
 * time: 4.7190001010894775
   631     1.168877e-01     6.641938e-01
 * time: 4.728000164031982
   632     1.168871e-01     4.957944e-01
 * time: 4.737000226974487
   633     1.168844e-01     4.572123e-01
 * time: 4.745000123977661
   634     1.168534e-01     1.215054e+00
 * time: 4.753000020980835
   635     1.168304e-01     9.967601e-01
 * time: 4.753000020980835
   636     1.167706e-01     1.051025e+00
 * time

   743     1.131203e-01     1.418467e+00
 * time: 5.564000129699707
   744     1.130750e-01     1.418464e+00
 * time: 5.572000026702881
   745     1.130485e-01     1.140752e+00
 * time: 5.581000089645386
   746     1.130212e-01     1.109290e+00
 * time: 5.589000225067139
   747     1.129898e-01     9.818487e-01
 * time: 5.5970001220703125
   748     1.129674e-01     9.476013e-01
 * time: 5.606000185012817
   749     1.129574e-01     1.037344e+00
 * time: 5.606000185012817
   750     1.129508e-01     7.681526e-01
 * time: 5.614000082015991
   751     1.129345e-01     1.012838e+00
 * time: 5.622000217437744
   752     1.129287e-01     7.667117e-01
 * time: 5.630000114440918
   753     1.129244e-01     7.663882e-01
 * time: 5.639000177383423
   754     1.129126e-01     8.472262e-01
 * time: 5.646000146865845
   755     1.129031e-01     1.042168e+00
 * time: 5.65500020980835
   756     1.128914e-01     5.172904e-01
 * time: 5.663000106811523
   757     1.128864e-01     9.281172e-01
 * time

   864     1.115701e-01     1.180771e+00
 * time: 6.476999998092651
   865     1.115359e-01     1.264695e+00
 * time: 6.485000133514404
   866     1.115320e-01     1.364098e+00
 * time: 6.494000196456909
   867     1.115195e-01     7.966855e-01
 * time: 6.502000093460083
   868     1.115142e-01     7.709944e-01
 * time: 6.502000093460083
   869     1.115023e-01     9.298393e-01
 * time: 6.513000011444092
   870     1.114849e-01     8.638489e-01
 * time: 6.51800012588501
   871     1.114764e-01     1.220745e+00
 * time: 6.526000022888184
   872     1.114313e-01     1.006274e+00
 * time: 6.5340001583099365
   873     1.114114e-01     1.083938e+00
 * time: 6.543000221252441
   874     1.113735e-01     1.491332e+00
 * time: 6.551000118255615
   875     1.113390e-01     1.592798e+00
 * time: 6.559000015258789
   876     1.113205e-01     1.416777e+00
 * time: 6.567000150680542
   877     1.112928e-01     1.037534e+00
 * time: 6.582000017166138
   878     1.112851e-01     7.807502e-01
 * time

   985     1.105651e-01     4.051566e-01
 * time: 7.390000104904175
   986     1.105633e-01     3.025057e-01
 * time: 7.390000104904175
   987     1.105624e-01     7.309687e-01
 * time: 7.39900016784668
   988     1.105615e-01     8.424802e-01
 * time: 7.4070000648498535
   989     1.105611e-01     8.855275e-01
 * time: 7.416000127792358
   990     1.105611e-01     8.855275e-01
 * time: 7.424000024795532
   991     1.105583e-01     5.759931e-01
 * time: 7.432000160217285
   992     1.105557e-01     2.909521e-01
 * time: 7.445000171661377
   993     1.105556e-01     3.431144e-01
 * time: 7.449000120162964
   994     1.105556e-01     3.466017e-01
 * time: 7.458000183105469
   995     1.105556e-01     3.462420e-01
 * time: 7.466000080108643
   996     1.105554e-01     7.855779e-01
 * time: 7.4740002155303955
   997     1.105515e-01     5.320501e-01
 * time: 7.484000205993652
   998     1.105487e-01     2.261366e-01
 * time: 7.491000175476074
   999     1.105487e-01     2.180660e-01
 * tim

40-element Vector{Float64}:
  0.19427824363001317
 -0.20479000779859027
 -0.3767030394884509
 -0.22274977367900686
  0.22255084348872428
 -0.37165685091889966
 -0.03213845283574842
 -0.16166691456500842
  0.13481305122150594
  0.21163665191774853
 -0.30679402113852994
 -0.38086500214495567
  0.09551740403146994
  ⋮
 -0.297432619930237
  0.2810224284029397
  0.13187678506917078
 -0.22356376889260737
 -0.2786469133928871
  0.00913900718082935
 -0.08275363065644976
 -0.29677146923539055
  0.12125857465185252
  0.11090619447037321
  0.26779002546778763
 -0.10333680757141542

In [16]:
@show minval

minval = 0.10920405951822704


0.10920405951822704

In [4]:
### ---------------------- Benchmarking ---------------------- ### 
# check n = 7 qubit case 
n = 7; 
q = 2; 
d = 2; 
t = 1;
λ = partitions_into_q_parts(n,q);
μ = partitions_into_q_parts(2t,q);
ν = partitions_into_q_parts(2t,q);
vλ = size(λ)[1];
vμ = size(μ)[1];
vν = size(ν)[1];

# ruskai_codewords_c0 = [sqrt(15);0;-sqrt(7);0;sqrt(21);0;sqrt(21);0]/8;
# ruskai_codewords_c1 = [0;sqrt(21);0;sqrt(21);0;-sqrt(7);0;sqrt(15)]/8;
# codewords = [ruskai_codewords_c0;ruskai_codewords_c1];

In [ ]:
# Initial guess 
codeword_length = size(partitions_into_q_parts(n,q))[1]
num_var_params = d * codeword_length
x0 = rand(num_var_params)

@time rules1(n,q,d,x0)
@btime rules1($n,$q,$d,$x0)

@time rule4_5(n,q,d,t,x0,λ,μ,ν,vλ,vμ,vν)
@btime rule4_5($n,$q,$d,$t,$x0,$λ,$μ,$ν,$vλ,$vμ,$vν)

function rule4_5_loop(n,q,d,t,x0,λ,μ,ν,vλ,vμ,vν)
    for i in 1:10000000
        rule4_5(n,q,d,t,x0,λ,μ,ν,vλ,vμ,vν)
    end
end
rule4_5_loop(n,q,d,t,x0,λ,μ,ν,vλ,vμ,vν)
@profview rule4_5_loop(n,q,d,t,x0,λ,μ,ν,vλ,vμ,vν)

  0.037071 seconds (45.21 k allocations: 3.015 MiB, 99.83% compilation time)
  39.960 ns (0 allocations: 0 bytes)


In [11]:
### check symmetry question 
x0_0 = normalize(rand(8))

function state1gen(x0_0)
    num_el = length(x0_0)
    x0_1 = zeros(num_el)
    for i in eachindex(x0_0)
        x0_1[i] = (-1)^(i+1) * x0_0[num_el-i+1]
    end 
    return x0_1 
end
x0_1 = state1gen(x0_0)


rules1(n,q,d,[x0_0;x0_1])
rule4_5(n,q,d,t,[x0_0;x0_1],λ,μ,ν,vλ,vμ,vν)

1.623743525724887